<a href="https://colab.research.google.com/github/polxo00/Algoritmos-de-optimizacion/blob/main/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>


Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
Número de actores: 10
Número de tomas : 30
Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario

(*) La respuesta es obligatoria





                                        

In [7]:
import pandas as pd

sheet_id = "1XVqBLOFDTk73qswb1fS4e2KUJ6N_GNzeDUW1Jy4G1vM"
sheet_name = "Sheet1"  # O canvia-ho segons el teu nom de pestanya

csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_raw = pd.read_csv(csv_url, header=None)

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?




Para calcular cuantas posibilidades hay sin tener en cuenta ninguna restricción, es decir que cada una de las tomas puede grabarse en cuaquier día de grabación, sin importar los actores que aparecen o el numero maximo de tomas que pueden hacer en un dia.

Esto es equivalente a partir el conjunto de las 30 tomas en grupos no vacios. Y este numero de particiones se conoce como el número de Bell, que para n = 30 es ???????.

10293358946226376485095653


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Inicialmente tome como estructura de datos un dataframe de pandas con toda la información, y en segunda instancia vi que la ulitma fila contenia datos diferentes y que por lo tanto lo devia separar en dos dataframes distintos.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

La función objetivo es minimizar el gasto total en actores de doblaje, devido a que cada actor cobra una cantidad fija independientemente del número de tomas grabadas en el día. Por lo tanto, si conseguimos optimizar para que los actores vengan el mínimo de dias possibles, vamos a reducir el gasto.

Por este motivo, se trata de un problema de minimización, ya que queremos reducir al mínimo el número de dias que deve asistir cada actor a la gravación, y en conseqüencia, reducir el coste.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [31]:
import numpy as np

df = df_raw.iloc[2:32, 1:11].copy()

df = df.astype(int)

tomas_actores = df.to_numpy()
actor_toma = tomas_actores.T
print(actor_toma.shape)
print(actor_toma)

(10, 30)
[[1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1]
 [1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0]
 [1 1 0 0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1]
 [1 1 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]]


In [41]:


def particions_valides(tomes, max_tomes_per_dia):
    if not tomes:
        yield []
        return
    for i in range(1, min(max_tomes_per_dia, len(tomes)) + 1):
        primer_grup = tomes[:i]
        resta = tomes[i:]
        for part_rest in particions_valides(resta, max_tomes_per_dia):
            yield [primer_grup] + part_rest

def cost_particio(particio, actor_toma):
    num_actors = actor_toma.shape[0]
    dies_actor = [0] * num_actors

    for dia in particio:
        for actor in range(num_actors):
            if any(actor_toma[actor][toma] == 1 for toma in dia):
                dies_actor[actor] += 1

    return sum(dies_actor)

tomes = list(range(3))
millor_cost = float(len(tomes)*10 +1) #En el peor de los casos, devemos hacer una escena al dia con todos los actores disponibles
millor_particio = None

for p in particions_valides(tomes, max_tomes_per_dia=6):

    cost = cost_particio(p, actor_toma)
    print(p)
    print(cost)
    if cost < millor_cost:
        millor_cost = cost
        millor_particio = p

print("Millor partició:", millor_particio)
print("Cost total:", millor_cost)

[[0], [1], [2]]
11
[[0], [1, 2]]
10
[[0, 1], [2]]
8
[[0, 1, 2]]
6
Millor partició: [[0, 1, 2]]
Cost total: 6


20.0

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta